# Введение
Цель данного ноутбука заключается в создании модели, способной отличать кликбейтные тексты от обычных.

# Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import warnings
warnings.filterwarnings('ignore')

2024-04-08 22:21:25.331665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Загружаем данные
Мы рассматриваем 3 датасета *(один с kaggle, другие два сгенерированы с помощью YandexGPT)*

## Датасеты сгенерированные с помощью YandexGPT
Они были сгенерированы в разное время, поэтому их структура немного отличается друг от друга.

In [3]:
FILE_1 = 'files/yandexgpt_generated_1.csv'
FILE_2 = 'files/yandexgpt_generated_2.csv'

temp1 = pd.read_csv(FILE_1, index_col='index')
temp2 = pd.read_csv(FILE_2, index_col='index')

temp1 = temp1[['title', 'clickbait_title']]
temp2 = temp2[['title', 'clickbait']]

temp2.rename(columns={'clickbait':'clickbait_title'}, inplace=True)

gpt_generated = pd.concat([temp1, temp2])

In [4]:
gpt_generated.sample(10)

,title,clickbait_title
index,,
280,Рэпер Tekashi69 сдал подельников по наркобизне...,Вот несколько вариантов заголовков для этой ст...
3788,Мужчина заказал еду домой и обнаружил неожидан...,"«Курьер съел ваш заказ! Что делать, если вы ок..."
3007,В России решили привлечь заключенных к стройка...,«Заключённые на стройках в Арктике: новый пово...
2657,Мчащийся на самокате по Крымскому мосту мужчин...,"К сожалению, я не могу ничего сказать об этом...."
4197,Анна Калашникова раскрыла траты на услуги ЖКХ,«Сколько тратят на коммуналку российские звёзд...
585,Памятник «Аленке» напугал жителей российского ...,"«Памятник ""Алёнке"" — это новое слово в искусст..."
975,Опубликован список самых обсуждаемых фильмов и...,"«Это изменит вашу жизнь! Узнайте, какие фильмы..."
696,Максим Галкин построил в своем замке SPA-центр,«Максим Галкин открыл свой SPA-центр: что скры...
2227,Отстраненный на десять лет российский футбольн...,«Скандал в российском футболе: судья Лапочкин ...


#### Объединим 2 колонки в одну

In [5]:
not_clickbait = pd.DataFrame({'title': gpt_generated['title']})
not_clickbait['is_clickbait'] = 0

clickbait = pd.DataFrame({'title': gpt_generated['clickbait_title']})
clickbait['is_clickbait'] = 1

df = pd.concat([clickbait, not_clickbait])

In [6]:
df.sample(5)

,title,is_clickbait
index,,
2114,«Катастрофа на Камчатке: 50 000 человек остали...,1
462,Хореограф Трусовой оценил риск исполнения четв...,0
1248,«Запрет банковских комиссий при оплате услуг Ж...,1
2799,"К сожалению, я не могу ничего сказать об этом....",1
4961,«Хабиб Нурмагомедов — новый король смешанных е...,1


## Датасет с Kaggle



In [8]:
FILE_3 = 'files/kaggle.csv'
kaggle = pd.read_csv(FILE_3, engine='python', sep=';')
kaggle.sample(5)

,titles,target
1805,России нужно сосредоточиться на внутренней пов...,0
2872,К делегации России на саммите АТЭС относятся д...,0
530,Долгожданный рост детских пособий: какие измен...,1
2898,Байден и Си Цзиньпин прогулялись после перегов...,0
152,"Назван самый полезный овощ, который нужно есть...",1


In [9]:
kaggle.rename(columns={'titles' : 'title', 'target' : 'is_clickbait'}, inplace=True)
df = pd.concat([kaggle, df])

In [10]:
df.sample(10)

,title,is_clickbait
4916,"К сожалению, я не могу ничего сказать об этом....",1
596,"«Громкий уход из ""Спартака"": откровения Дениса...",1
1400,Мбаппе стал новым капитаном сборной Франции по...,0
4885,Тренер сборной России назвал условие возможног...,0
1932,«Латвийский голкипер Янис Калниньш: новый пово...,1
295,В европейское жилье расхотели вкладывать деньги,0
1415,«Гром среди ясного неба!»: Алина Загитова уход...,1
1659,**Сон Хын Мин — новый король футбола! Как ему ...,1
1008,Александр Мостовой станет главным тренером? Чт...,1
928,«Шокирующая правда об автомобилях: более полут...,1


# Очистим данные

In [11]:
# Удалим дубликаты и пропуски
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Удалим заголовки, на которые YandexGPT не дал ответы
no_answer = 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?'
df = df[~df['title'].str.startswith(no_answer)]

# Удалим ненужную информацию из заголовков
def clean(text):
    text = text.strip()
    # Фраза предлагающая один из вариантов кликбейта
    if text.startswith('Вот один из'):
        parts = [part.strip() for part in text.split(':')[1:]]
        text = ' '.join(parts)
    # Фраза предлагающая несколько вариантов кликбейта
    if text.startswith('Вот несколько'):
        text = text.split(':')[1].strip()
    # Ненужные знаки при генерации текста
    useless = ['«', '»', '**', '*']
    for to_change in useless:
        text = text.replace(to_change, '')
    return text

df['title'] = df['title'].apply(clean)

In [12]:
# Баланс классов после очистки
df['is_clickbait'].value_counts()

is_clickbait
0    7529
1    6395
Name: count, dtype: int64

In [13]:
# Посмотрим 10 прозвольных заголовков после очистки
for x in df.sample(10)['title']:
    print(x)

Порнозвезда с большим опытом назвала свое секретное оружие
Отправку оружия Израилю поддерживают треть американцев, показал опрос
Пять украинских компаний подписали контракты с Газпромом
Сенсационный взлёт России в таблице УЕФА: всего две победы, и мы уже на девятом месте!
Туалет-призрак: архитекторы из Токио создали нечто невероятное!
Не такой, как другие дети: всплыла неожиданная правда о сыне Королевой и Глушко
Как открыть примёрзшую дверь: уникальный способ, который спасёт ваш автомобиль
Греция и Кипр — лучшие места для отпуска: власти оплатят лечение от коронавируса!
Россиян предупредили о жаре почти до 40 градусов
Сбербанк облегчил оформление ипотеки


# Разбиваем данные

In [14]:
text = df['title'].values
labels = df['is_clickbait'].values
text_train, text_test, y_train, y_test = train_test_split(text, labels, test_size=0.2)
text_train, train_val, y_train, y_val = train_test_split(text_train, y_train, test_size=0.15)

print(f'Train: X~{text_train.shape[0]}, y~{y_train.shape[0]}')
print(f'Test: X~{text_test.shape[0]}, y~{y_test.shape[0]}')
print(f'Validation: X~{train_val.shape[0]}, y~{y_val.shape[0]}')

Train: X~9468, y~9468
Test: X~2785, y~2785
Validation: X~1671, y~1671


# Токенизация

In [15]:
vocab_size = 5000
maxlen = 100
embedding_size = 32

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text)

X_train = tokenizer.texts_to_sequences(text_train)
x_test = tokenizer.texts_to_sequences(text_test)
x_val = tokenizer.texts_to_sequences(train_val)

X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

# Обучение модели

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(LSTM(32, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

2024-04-08 22:23:53.176308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100, 32)           8320      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 168,353
Trainable params: 168,353
Non-trainable params: 0
__________________________________________________

In [17]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=3,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='weights.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=64, validation_data=(x_val, y_val), epochs=10, callbacks=callbacks)

Epoch 1/10
147/148 [============================>.] - ETA: 0s - loss: 0.5700 - accuracy: 0.7174
Epoch 1: val_accuracy improved from -inf to 0.89767, saving model to weights.h5
148/148 [==============================] - 9s 49ms/step - loss: 0.5684 - accuracy: 0.7186 - val_loss: 0.3521 - val_accuracy: 0.8977
Epoch 2/10
147/148 [============================>.] - ETA: 0s - loss: 0.2737 - accuracy: 0.9239
Epoch 2: val_accuracy improved from 0.89767 to 0.92101, saving model to weights.h5
148/148 [==============================] - 7s 45ms/step - loss: 0.2731 - accuracy: 0.9242 - val_loss: 0.2446 - val_accuracy: 0.9210
Epoch 3/10
148/148 [==============================] - ETA: 0s - loss: 0.1757 - accuracy: 0.9538
Epoch 3: val_accuracy improved from 0.92101 to 0.92340, saving model to weights.h5
148/148 [==============================] - 8s 55ms/step - loss: 0.1757 - accuracy: 0.9538 - val_loss: 0.2053 - val_accuracy: 0.9234
Epoch 4/10
148/148 [==============================] - ETA: 0s - loss: 

# Проверка модели

In [19]:
predictions_probability = model.predict(x_test)
predictions = [round(x[0]) for x in predictions_probability] # округляем до 0 или 1
print(classification_report(y_test, predictions))

88/88 [==============================] - 1s 9ms/step
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1498
           1       0.91      0.90      0.91      1287

    accuracy                           0.92      2785
   macro avg       0.92      0.92      0.92      2785
weighted avg       0.92      0.92      0.92      2785

